In [3]:
import praw
import json
import time

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT")


In [ ]:
import os
import json
import time
from dotenv import load_dotenv
import praw

# ==== 🔒 载入环境变量 ====
load_dotenv()
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT")

# ==== ✅ 初始化 Reddit 实例 ====
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# ==== 🗂️ 子版块 & 关键词 ====
subreddits = ["Coffee", "espresso", "barista", "pourover"]

keywords = {
    "flavor": ["bitter", "sweet", "acidity", "fruity", "nutty", "creamy"],
    "type": ["espresso", "latte", "cold brew", "americano"],
    "brew": ["french press", "pour-over", "moka pot", "aeropress"],
    "origin": ["arabica", "robusta", "ethiopia", "colombia"]
}

# ==== 🚀 主函数 ====
def collect_sentences_by_keyword(keywords_dict, max_posts=300, max_comments=10):
    result = []

    for category, kw_list in keywords_dict.items():
        for keyword in kw_list:
            print(f"\n🔍 Searching for: '{keyword}' ({category})")
            keyword_data = {
                "keyword": keyword,
                "category": category,
                "examples": [],
                "size": 0
            }

            matched = set()
            count = 0

            for sub in subreddits:
                print(f"  📂 Subreddit: r/{sub}")
                try:
                    for post in reddit.subreddit(sub).search(keyword, limit=max_posts, sort='relevance'):
                        post.comments.replace_more(limit=0)
                        for comment in post.comments[:max_comments]:
                            text = comment.body.strip()
                            if keyword.lower() in text.lower() and len(text) >= 15:
                                if text not in matched:
                                    keyword_data["examples"].append(text)
                                    matched.add(text)
                                    count += 1
                                    if count >= 20:  # 最多保留20条
                                        break
                        if count >= 20:
                            break
                    if count >= 20:
                        break
                except Exception as e:
                    print(f"⚠️ Error in r/{sub}: {e}")
                    continue
                time.sleep(0.2)

            keyword_data["size"] = len(keyword_data["examples"])
            result.append(keyword_data)
            print(f"✅ Collected {keyword_data['size']} comments for '{keyword}'")

    return result

# ==== 📦 保存结果 ====
all_data = collect_sentences_by_keyword(keywords)

with open("reddit_coffee_bubbles_2.json", "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)

print("\nScrapping saved, data saved as reddit_coffee_bubbles_2.json")



🔍 Searching for: 'bitter' (flavor)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'bitter'

🔍 Searching for: 'sweet' (flavor)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'sweet'

🔍 Searching for: 'acidity' (flavor)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'acidity'

🔍 Searching for: 'fruity' (flavor)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'fruity'

🔍 Searching for: 'nutty' (flavor)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'nutty'

🔍 Searching for: 'creamy' (flavor)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'creamy'

🔍 Searching for: 'espresso' (type)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'espresso'

🔍 Searching for: 'latte' (type)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'latte'

🔍 Searching for: 'cold brew' (type)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'cold brew'

🔍 Searching for: 'americano' (type)
  📂 Subreddit: r/Coffee
✅ Collected 20 comments for 'americano'

🔍 Searching for: 'french 

In [9]:
import os
import json
import time
from dotenv import load_dotenv
import praw

# ==== 读取密钥 ====
load_dotenv()
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT")

# ==== 初始化 Reddit ====
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

# ==== 关键词 + subreddit ====
keywords = {
    "flavor": ["bitter", "sweet", "acidity", "fruity", "nutty", "creamy"],
    "type": ["espresso", "latte", "cold brew", "americano"],
    "brew": ["french press", "pour-over", "moka pot", "aeropress"],
    "origin": ["arabica", "robusta", "ethiopia", "colombia"]
}

subreddits = ["Coffee", "espresso", "barista", "pourover", "coffeegear", "homebarista"]

# ==== 主函数 ====
def collect_all_comments(keywords_dict, max_posts=500, max_comments=20):
    collected = []
    matched = set()

    for category, kw_list in keywords_dict.items():
        for keyword in kw_list:
            print(f"\nKeyword: {keyword} ({category})")
            for sub in subreddits:
                print(f"  Subreddit: r/{sub}")
                try:
                    posts = reddit.subreddit(sub).search(keyword, limit=max_posts, sort="relevance")
                    for post in posts:
                        post_url = f"https://www.reddit.com{post.permalink}"
                        post.comments.replace_more(limit=0)
                        for comment in post.comments[:max_comments]:
                            text = comment.body.strip()
                            if keyword.lower() in text.lower() and len(text) >= 15:
                                key = (keyword, text)
                                if key not in matched:
                                    matched.add(key)
                                    collected.append({
                                        "keyword": keyword,
                                        "category": category,
                                        "comment": text,
                                        "subreddit": sub,
                                        "score": comment.score,
                                        "post_url": post_url
                                    })
                    time.sleep(1.0)
                except Exception as e:
                    print(f"Error in r/{sub}: {e}")
                    continue

    return collected

# ==== 抓取并保存 ====
all_comments = collect_all_comments(keywords)
print(f"\nTotal Number of Scrapped Posts：{len(all_comments)}")

with open("reddit_coffee_sentences.json", "w", encoding="utf-8") as f:
    json.dump(all_comments, f, ensure_ascii=False, indent=2)



Keyword: bitter (flavor)
  Subreddit: r/Coffee
  Subreddit: r/espresso
  Subreddit: r/barista
  Subreddit: r/pourover
  Subreddit: r/coffeegear
Error in r/coffeegear: Redirect to /subreddits/search
  Subreddit: r/homebarista

Keyword: sweet (flavor)
  Subreddit: r/Coffee
Error in r/Coffee: received 429 HTTP response
  Subreddit: r/espresso
Error in r/espresso: received 429 HTTP response
  Subreddit: r/barista
Error in r/barista: received 429 HTTP response
  Subreddit: r/pourover
Error in r/pourover: received 429 HTTP response
  Subreddit: r/coffeegear
Error in r/coffeegear: received 429 HTTP response
  Subreddit: r/homebarista
Error in r/homebarista: received 429 HTTP response

Keyword: acidity (flavor)
  Subreddit: r/Coffee
Error in r/Coffee: received 429 HTTP response
  Subreddit: r/espresso
Error in r/espresso: received 429 HTTP response
  Subreddit: r/barista
Error in r/barista: received 429 HTTP response
  Subreddit: r/pourover
Error in r/pourover: received 429 HTTP response
  Su

KeyboardInterrupt: 